# Assessing the number of transition options

Here we calculate the number of transition options for the 1627 occupations presented in the Mapping Career Causeways report.

# 0. Import dependencies and inputs

In [1]:
%run ../notebook_preamble_Transitions.ipy
import os

data = load_data.Data()

## 0.1 Generate 'filtering' matrices

In [3]:
file_name = 'filter_matrices_Report_occupations.pickle'
export_path = f'{data_folder}interim/transitions/{file_name}'

if os.path.exists(export_path):
    filter_matrices = pickle.load(open(export_path,'rb'))
    print(f'Imported filtering matrices from {export_path}')
else:
    # May take about 30 mins
    filter_matrices = trans_utils.create_filtering_matrices(
        origin_ids='report', 
        destination_ids= 'report',
        export_path = export_path)

Imported filtering matrices from /Users/karliskanders/Documents/mapping-career-causeways/codebase/data/interim/transitions/filter_matrices_Report_occupations.pickle


In [4]:
# All the different filtering matrices, and the identities of their rows (rows='origin_ids' & cols='destination_ids')
for key in filter_matrices.keys():
    print(key)

F_viable
F_min_viable
F_highly_viable
F_desirable
F_jobzone
F_earnings
F_not_high_risk
F_safer
F_strictly_safe
F_not_self
F_safe_desirable
F_strictly_safe_desirable
origin_ids
destination_ids


# 1. Calculate the number of transitions for each occupation

## 1.1 Number of desirable transitions

First, we assess transitions that are desirable, meaning that they are both viable (sufficiently good fit in terms of skills, work activities and work context) and do not incure a major salary loss.


In [77]:
# Desirable transitions
n_desirable = np.sum(filter_matrices['F_desirable'], axis=1)

# Highly viable, desirable transitions
n_desirable_and_highly_viable = np.sum(
    (filter_matrices['F_desirable'] & filter_matrices['F_highly_viable']), axis=1)

## 1.2 Number of safe and desirable transitions (default condition)

We use the default criteria for safe transitions - that safe destination is defined as not being 'high risk'. We use these results to compare 'high risk' versus 'lower risk' occupations ('lower risk' occupations belonging to the 'other' or 'low risk' categories).

In [78]:
# Safe and desirable transitions
n_safe_desirable = np.sum(filter_matrices['F_safe_desirable'], axis=1)

# Highly viable, safe and desirable transitions
n_safe_desirable_and_highly_viable = np.sum(
    (filter_matrices['F_safe_desirable'] & filter_matrices['F_highly_viable']), axis=1)


## 1.3 Number of safe and desirable transitions (strict condition)

We use the more stricter criteria for defining safe transitions when making comparisons solely between 'high risk' occupations.

In [79]:
# Strictly safer and desirable transitions
n_safe_desirable_strict = np.sum(filter_matrices['F_strictly_safe_desirable'], axis=1)

# Highly viable, strictly safe
n_safe_desirable_strict_and_highly_viable = np.sum(
    (filter_matrices['F_strictly_safe_desirable'] & filter_matrices['F_highly_viable']), axis=1)


## 1.4 Export the transition option results

In [123]:
occ_transitions = data.occ_report[['id', 'concept_uri', 'preferred_label', 'isco_level_4', 'risk_category']].copy()
occ_transitions['n_desirable'] = n_desirable
occ_transitions['n_desirable_and_highly_viable'] = n_desirable_and_highly_viable
occ_transitions['n_safe_desirable'] = n_safe_desirable
occ_transitions['n_safe_desirable_and_highly_viable'] = n_safe_desirable_and_highly_viable
occ_transitions['n_safe_desirable_strict'] = n_safe_desirable_strict
occ_transitions['n_safe_desirable_strict_and_highly_viable'] = n_safe_desirable_strict_and_highly_viable

In [125]:
# Rename the risk categories, as used in the report
occ_transitions.loc[occ_transitions.risk_category.isin(['Low risk', 'Other']), 'risk_category'] = 'Lower risk'

In [129]:
occ_transitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1627 entries, 0 to 1626
Data columns (total 11 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   id                                         1627 non-null   int64 
 1   concept_uri                                1627 non-null   object
 2   preferred_label                            1627 non-null   object
 3   isco_level_4                               1627 non-null   int64 
 4   risk_category                              1627 non-null   object
 5   n_desirable                                1627 non-null   int64 
 6   n_desirable_and_highly_viable              1627 non-null   int64 
 7   n_safe_desirable                           1627 non-null   int64 
 8   n_safe_desirable_and_highly_viable         1627 non-null   int64 
 9   n_safe_desirable_strict                    1627 non-null   int64 
 10  n_safe_desirable_strict_and_highly_v

In [127]:
occ_transitions.groupby('risk_category').agg({'id': 'count'})

,id
risk_category,
High risk,217
Lower risk,1410


In [128]:
# Save
occ_transitions.to_csv(data_folder + 'processed/transitions/ESCO_number_of_transitions.csv', index=False)
